
# Workflow Practice

In this notebook, you’ll practice connecting to a SQLite database, creating tables from CSV files using Pandas, and writing SQL queries to explore the data.

The dataset comes from the [Bike Store Sample Database](https://www.kaggle.com/datasets/dillonmyrick/bike-store-sample-database) by Dillon Myrick. It models a fictional bike retailer with multiple stores, products, customers, and staff. Each table connects to others using foreign keys such as `customer_id`, `store_id`, and `product_id`.

You’ll:
- Connect to a local SQLite database
- Create tables using `pandas.to_sql()`
- Write and test SQL queries using `pd.read_sql()`

All of your work will take place directly in this notebook. Each question prompt is written below as a Markdown cell, followed by an empty code cell for you to write your query.



## Step 1: Connect to the Database

Run the following cell to connect to (or create) a SQLite database called `bike_store.db`.  
If the file doesn’t exist yet, SQLite will automatically create it.


In [24]:
import sqlite3
import pandas as pd

In [25]:
connection = sqlite3.connect("bike_store.db")
connection


## Step 2: Create Tables from CSV Files

The `data/` folder contains one CSV file per table.  
Use `pandas.read_csv()` and `DataFrame.to_sql()` to load each file into your database.

You only need to do this once.  
After that, you’ll be able to run queries against your newly created tables.


In [26]:
# Example for one file
customers = pd.read_csv("data/customers.csv")
customers.to_sql("customers", connection, if_exists="replace", index=False)

1445

In [27]:
customers.head()

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,NaN,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,NaN,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,NaN,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,NaN,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


In [28]:
# Repeat for all other files in the data folder, or use a loop.
files = [
    "customers",
    "orders",
    "order_items",
    "products",
    "categories",
    "stores",
    "staffs",
    "brands"
]

for name in files:
    df = pd.read_csv(f"data/{name}.csv")
    df.to_sql(name, connection, if_exists="replace", index=False)
    print(f"Loaded {name}: {len(df)} rows")

Loaded customers: 1445 rows
Loaded orders: 1615 rows
Loaded order_items: 4722 rows
Loaded products: 321 rows
Loaded categories: 7 rows
Loaded stores: 3 rows
Loaded staffs: 10 rows
Loaded brands: 9 rows


### Verify Your Tables

Run a query to make sure your tables were created successfully.

In [29]:

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)


,name
0,customers
1,orders
2,order_items
3,products
4,categories
5,stores
6,staffs
7,brands


## Step 3: Test a Simple Query

Before starting the exercises, confirm your connection and tables are working by previewing the first few rows of the `customers` table.

In [30]:

pd.read_sql("SELECT discount from order_items", connection)


,discount
0,0.20
1,0.07
2,0.05
3,0.05
4,0.20
...,...
4717,0.07
4718,0.20
4719,0.20
4720,0.07


In [31]:
# Customer Preview Query 
pd.read_sql("""
SELECT *
FROM customers
LIMIT 5;
""", connection)

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,None,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,None,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,None,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,None,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


### Q1. List all customers and their cities.

Return the first name, last name, and city of each customer. Sort alphabetically by last name and then by first name.

In [32]:
# Listing all customers with city info
# Sorted by last then first name, alphabetical order)
query1 = """
SELECT first_name, last_name, city
FROM customers
ORDER BY last_name ASC, first_name ASC; 
"""
pd.read_sql(query1, connection)

,first_name,last_name,city
0,Ester,Acevedo,San Lorenzo
1,Jamika,Acevedo,Ozone Park
2,Penny,Acevedo,Ballston Spa
3,Bettyann,Acosta,Lancaster
4,Shery,Acosta,Saratoga Springs
...,...,...,...
1440,Edda,Young,North Tonawanda
1441,Jasmin,Young,Helotes
1442,Alexandria,Zamora,Schenectady
1443,Jayme,Zamora,Springfield Gardens


### Q2. Show all products and their prices.

Display each product name along with its list price. Sort by price in descending order.

In [33]:
# Show all products & prices
# sort by price DESC
query2 = """
SELECT product_name, list_price
FROM products
ORDER BY list_price DESC; 
"""
pd.read_sql(query2, connection)

,product_name,list_price
0,Trek Domane SLR 9 Disc - 2018,11999.99
1,Trek Domane SLR 8 Disc - 2018,7499.99
2,Trek Silque SLR 8 Women's - 2017,6499.99
3,Trek Domane SL Frameset - 2018,6499.99
4,Trek Domane SL Frameset Women's - 2018,6499.99
...,...,...
316,Trek Kickster - 2018,159.99
317,Trek Boy's Kickster - 2015/2017,149.99
318,Trek Girl's Kickster - 2017,149.99
319,Sun Bicycles Lil Kitt'n - 2017,109.99


### Q3. Find all customers from California.

Return first name, last name, city, and state for all customers whose state is 'CA'. Sort alphabetically by last name.

In [34]:
# Find all CA customers -- info: name (first, last), city, state for CA CXs.
# Sort alphabetically (by last_name)
query3 = """ 
SELECT first_name, last_name, city, state
FROM customers
WHERE state = "CA"
ORDER BY last_name ASC;
"""

pd.read_sql(query3, connection)

,first_name,last_name,city,state
0,Ester,Acevedo,San Lorenzo,CA
1,Jamaal,Albert,Torrance,CA
2,Sindy,Anderson,Pomona,CA
3,Twana,Arnold,Anaheim,CA
4,Selene,Austin,Duarte,CA
...,...,...,...,...
279,Darren,Witt,Coachella,CA
280,Lucy,Woods,Palos Verdes Peninsula,CA
281,Joel,Wynn,San Diego,CA
282,Yvone,Yates,San Pablo,CA


### Q4. Count how many products are in each category.

Return the category name and the number of products in that category. Sort from the highest count to the lowest.

In [35]:
# Sort product category and count (highest to lowest)
query4 = """ 
SELECT 
    categories.category_name,
    COUNT(products.product_id) AS product_count
FROM products
JOIN categories
    ON products.category_id = categories.category_id
GROUP BY categories.category_name
ORDER BY product_count DESC;
"""
pd.read_sql(query4, connection)

,category_name,product_count
0,Cruisers Bicycles,78
1,Road Bikes,60
2,Mountain Bikes,60
3,Children Bicycles,59
4,Comfort Bicycles,30
5,Electric Bikes,24
6,Cyclocross Bicycles,10


### Q5. Find all orders placed in 2018.

List the order ID, order date, and customer ID for orders made during the year 2018. Sort by order date.

In [36]:
# Find all 2018 orders, sort by order_date
query5 = """ 
SELECT order_id, order_date, customer_id
FROM orders
WHERE order_date >= '2018-01-01'
    AND order_date < '2019-01-01'
ORDER BY order_date ASC;
"""
pd.read_sql(query5, connection)

,order_id,order_date,customer_id
0,1324,2018-01-01,862
1,1325,2018-01-01,68
2,1326,2018-01-01,567
3,1327,2018-01-02,1026
4,1328,2018-01-02,1083
...,...,...,...
287,1611,2018-09-06,6
288,1612,2018-10-21,3
289,1613,2018-11-18,1
290,1614,2018-11-28,135


### Q6. Show each order with its total number of items.

Join the `orders` and `order_items` tables. Group by order ID and return the number of items per order.

In [37]:
# show each order w/total # of items
query6 = """
SELECT 
    orders.order_id,
    COUNT(order_items.order_id) AS total_items
FROM orders
JOIN order_items
    ON orders.order_id = order_items.order_id
GROUP BY orders.order_id
ORDER BY orders.order_id ASC;
"""
pd.read_sql(query6, connection)

,order_id,total_items
0,1,5
1,2,2
2,3,2
3,4,1
4,5,3
...,...,...
1610,1611,3
1611,1612,5
1612,1613,2
1613,1614,3


### Q7. List total revenue per store.

Revenue = quantity * list_price * (1 - discount). Join `orders`, `order_items`, and `stores`, group by store name, and return total revenue.

In [38]:
# list - total revenue per store
query7 = """ 
SELECT
  stores.store_name,
  SUM(order_items.quantity * order_items.list_price * (1 - order_items.discount)) AS total_revenue
FROM orders
JOIN order_items
  ON orders.order_id = order_items.order_id
JOIN stores
  ON orders.store_id = stores.store_id
GROUP BY stores.store_name
ORDER BY total_revenue DESC;
"""
pd.read_sql(query7, connection)

,store_name,total_revenue
0,Baldwin Bikes,5.215751e+06
1,Santa Cruz Bikes,1.605823e+06
2,Rowlett Bikes,8.675422e+05


### Q8. Find the top 5 customers who spent the most overall.

Join `customers`, `orders`, and `order_items`. Sum the total spending per customer and return the top five spenders.

In [39]:
# Find the TOP 5 spenders
query8 = """ 
SELECT
    customers.customer_id,
    customers.first_name, 
    customers.last_name,
    SUM(order_items.quantity * order_items.list_price * (1 - order_items.discount)) AS total_spent
FROM customers
JOIN orders
    ON customers.customer_id = orders.customer_id
JOIN order_items
    ON orders.order_id = order_items.order_id
GROUP BY customers.customer_id, customers.first_name, customers.last_name
ORDER BY total_spent DESC
LIMIT 5;
"""
pd.read_sql(query8, connection)

,customer_id,first_name,last_name,total_spent
0,94,Sharyn,Hopkins,34807.9392
1,10,Pamelia,Newman,33634.2604
2,75,Abby,Gamble,32803.0062
3,6,Lyndsey,Bean,32675.0725
4,16,Emmitt,Sanchez,31925.8857


### Q9. Show the best-selling product in each category.

Join `products`, `order_items`, and `categories`. For each category, identify the product with the highest total quantity sold.

In [40]:
# Your query here
query9 = """ 
WITH product_totals AS (
    SELECT
        categories.category_name,
        products.product_name,
        SUM(order_items.quantity) AS total_quantity_sold
    FROM categories
    JOIN products
        ON categories.category_id = products.category_id
    JOIN order_items
        ON products.product_id = order_items.product_id
    GROUP BY
        categories.category_name,
        products.product_name
),
ranked_products AS (
    SELECT
        category_name,
        product_name,
        total_quantity_sold,
        ROW_NUMBER() OVER (
            PARTITION BY category_name
            ORDER BY total_quantity_sold DESC
        ) AS category_rank
    FROM product_totals
)
SELECT
    category_name,
    product_name,
    total_quantity_sold
FROM ranked_products
WHERE category_rank = 1
ORDER BY category_name;
"""
pd.read_sql(query9, connection)

,category_name,product_name,total_quantity_sold
0,Children Bicycles,Electra Girl's Hawaii 1 (20-inch) - 2015/2016,154
1,Comfort Bicycles,Electra Townie Original 7D - 2015/2016,148
2,Cruisers Bicycles,Electra Cruiser 1 (24-Inch) - 2016,157
3,Cyclocross Bicycles,Surly Straggler 650b - 2016,151
4,Electric Bikes,Trek Conduit+ - 2016,145
5,Mountain Bikes,Surly Ice Cream Truck Frameset - 2016,167
6,Road Bikes,Trek Domane SLR 6 Disc - 2017,43


### Q10. Identify the employees (staff) who processed the most orders.

Join `staffs` and `orders`. Count the number of orders handled by each staff member and return the results sorted by highest total.

In [41]:
# ID the EE (staff) who processed most orders
# Join staffs & orders, count orders per staff member, stor by highest rank
query10 = """ 
SELECT
    staffs.staff_id,
    staffs.first_name,
    staffs.last_name,
    COUNT(orders.order_id) AS total_orders_processed
FROM staffs
JOIN orders
    ON staffs.staff_id = orders.staff_id
GROUP BY
    staffs.staff_id,
    staffs.first_name,
    staffs.last_name
ORDER BY total_orders_processed DESC;
"""
pd.read_sql(query10, connection)

,staff_id,first_name,last_name,total_orders_processed
0,6,Marcelene,Boyer,553
1,7,Venita,Daniel,540
2,3,Genna,Serrano,184
3,2,Mireya,Copeland,164
4,8,Kali,Vargas,88
5,9,Layla,Terrell,86
